In [24]:
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
from src import utils, load_data
from src.models import sparse_maml
from tqdm.notebook import tqdm
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision
from torch.utils.tensorboard import SummaryWriter
from collections import namedtuple
from src import utils
from src.models.sparse_meta import SparseMeta

print(f"found gpu: {torch.cuda.is_available()}")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
found gpu: True


In [8]:
expr_params = {
    "seeds": {
        "torch": 222,
        "cuda": 222,
        "numpy": 222
    },
    "expr_id": utils.new_expr_id("lth maml"),
    "model_training_params": {
        "training_iterations": 60000, #epoch
        "n_way": 5,                        # number of classes to choose between for each task
        "k_spt": 1,                        # k shot for support set (number of examples per class per task)
        "k_qry": 15,                       # k shot for query set (number of examples per class per task)
        "imgsz": 84,                       # image size
        "imgc": 3,                         # this isn't used anywhere????? no idea what it does???? they say its supposed to be 1 or 3...
        "task_num": 4,                     # meta model batch size
        "meta_lr": 1e-3,                   # meta model learning rate
        "update_lr": 0.01,                 # task specific model learning rate
        "update_step": 5,                  # task specific model training epochs
        "update_step_test": 10,            # task specific model testing epochs
#         "optimizer": ("adam", {"lr": 0.0001}),
#         "loss_func": "cross_entropy",
        "lamda": 0.0001,
        "model_name": "Sparse MAML",
        "dataset_name": "mini_imagenet",
        "layer_definitions": None
    }
}

expr_params["model_training_params"]["layer_definitions"] = [
    ('conv2d', [32, 3, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 2, 0]),
    ('conv2d', [32, 32, 3, 3, 1, 0]),
    ('relu', [True]),
    ('bn', [32]),
    ('max_pool2d', [2, 1, 0]),
    ('flatten', []),
    ('linear', [expr_params["model_training_params"]["n_way"], 32 * 5 * 5]) # 32 * 5 * 5
]
utils.set_seeds(expr_params["seeds"])

In [9]:
args = expr_params["model_training_params"]
dataset = load_data.mini_imagenet(args)

shuffle DB :train, b:10000, 5-way, 1-shot, 15-query, resize:84
shuffle DB :val, b:100, 5-way, 1-shot, 15-query, resize:84
shuffle DB :test, b:100, 5-way, 1-shot, 15-query, resize:84


In [10]:
accs, best_model = sparse_maml.run(dataset, expr_params)

train epoch 0
Time 2020-12-08 19:43:58.570523
 step: 0 	training acc: [0.21       0.23333333 0.23333333 0.23333333 0.23333333 0.23333333]
validating model...
val acc: [0.2035 0.2413 0.2451 0.2448 0.244  0.2448 0.2448 0.2446 0.2444 0.2448
 0.2451]
 step: 30 	training acc: [0.16666667 0.27666667 0.28333333 0.28666667 0.28333333 0.28      ]
 step: 60 	training acc: [0.22666667 0.27666667 0.24666667 0.25       0.23666667 0.24      ]
 step: 90 	training acc: [0.19333333 0.29666667 0.3        0.33       0.33666667 0.34666667]
 step: 120 	training acc: [0.22666667 0.26333333 0.28666667 0.28666667 0.3        0.30666667]
 step: 150 	training acc: [0.19       0.31666667 0.32333333 0.34333333 0.34333333 0.35333333]
 step: 180 	training acc: [0.19       0.3        0.32       0.33       0.32666667 0.32333333]
 step: 210 	training acc: [0.2        0.26333333 0.29       0.29       0.3        0.30666667]
 step: 240 	training acc: [0.18       0.30333333 0.34       0.35666667 0.37       0.36333333]
 ste

In [11]:
utils.sparsity(best_model, 1e-3)

tensor(0.2704)

In [19]:
model = SparseMeta(expr_params["model_training_params"], expr_params["model_training_params"]["layer_definitions"])
model.load_state_dict(best_model)


<All keys matched successfully>

In [21]:
_, _, test_data = dataset

In [30]:
test_accs = sparse_maml.test(model, test_data)

In [31]:
test_accs

array([0.1957, 0.4026, 0.4214, 0.4224, 0.4253, 0.4263, 0.4287, 0.4285,
       0.4292, 0.429 , 0.43  ], dtype=float16)